In [1]:
from tqdm import tqdm
import pickle
import pandas as pd
import numpy as np
import torch
import requests

In [2]:
PICKLES_ADDRESS = '../data/pickles/'
LAG_DATA_ADDRESS = 'lag_data/'
PUBMED_ADDRESS = '../data/pubmed_dataset/'
SBIR_ADDRESS = '../data/sbir_dataset/'

In [3]:
def get_parent(dui):

    umls_api_key = ''
    base_url = 'https://uts-ws.nlm.nih.gov'
    
    # Search for MeSH mappings
    search_endpoint = '/rest/content/current/source/MSH/{dui}/parents'
    search_url = f'{base_url}{search_endpoint}'
    params = {
        'apiKey': umls_api_key,
    }
    response = requests.get(search_url.format(dui=dui), params=params)
    # print(response.status_code)
    result = response.json()
    parent_name, parent_dui = result['result'][0]['name'], result['result'][0]['ui']
    return parent_dui, parent_name


print("Machine Learning's parent:", get_parent('D000069550'))
print("Deep Learning's parent:", get_parent('D000077321'))

Machine Learning's parent: ('D001185', 'Artificial Intelligence')
Deep Learning's parent: ('D000069550', 'Machine Learning')


In [4]:
get_parent('U000017')

('V-MSH', 'Medical Subject Headings')

In [5]:
def get_ancestors(dui, trail_memory={}):
    current_dui = dui
    current_name = ''

    ancestors = []
    #iterate and get_parent at each step
    while True:
        new_dui, new_name = get_parent(current_dui)
        if new_name == 'Topical Descriptor':
            break
        # check if the new_dui is in the trail_memory
        if new_dui in trail_memory:
            pre_calculated_trail = trail_memory[new_dui][::-1]
            ancestors += pre_calculated_trail
            break
        else:
            ancestors.append(new_dui)
        current_dui, current_name = new_dui, new_name
    return ancestors[::-1] + [dui] # rest the input dui at last 


print(get_ancestors('D000077321'))
temp_memory = {'D001185':['U000010', 'D007254', 'D003205', 'D000465', 'D001185']}
print(get_ancestors('D000077321', temp_memory))

['U000010', 'D007254', 'D003205', 'D000465', 'D001185', 'D000069550', 'D000077321']
['U000010', 'D007254', 'D003205', 'D000465', 'D001185', 'D000069550', 'D000077321']


In [6]:
def get_name(dui):

    umls_api_key = ''
    base_url = 'https://uts-ws.nlm.nih.gov'
    
    # Search for MeSH mappings
    search_endpoint = '/rest/content/current/source/MSH/{dui}'
    search_url = f'{base_url}{search_endpoint}'
    params = {
        'apiKey': umls_api_key,
        'sabs': 'MSH',
    }
    response = requests.get(search_url.format(dui=dui), params=params)
    result = response.json()['result']['name']
    # parent_name, parent_dui = result['result'][0]['name'], result['result'][0]['ui']
    # return parent_dui, parent_name
    return result


get_name('D000069550')

'Machine Learning'

In [7]:
with open(PICKLES_ADDRESS + 'heading2dui.pkl', 'rb') as f:
    heading2dui = pickle.load(f)

dui2heading = {v:k for k,v in heading2dui.items()}
len(dui2heading)

29655

In [115]:
# go through each dui from dui2heading keys and get ancestors and make dui2trail where dui is key and anscestors list is the value
dui2trail = {}
c = 0
for dui in tqdm(list(dui2heading.keys())[:]):
    try:
        trail = get_ancestors(dui, dui2trail)
        for i, dui in enumerate(trail):
            trail_sliced = trail[:i+1]
            dui2trail[dui] = trail_sliced
        if c % 100 == 0:
            with open(PICKLES_ADDRESS + 'dui2trail.pkl', 'wb') as f:
                pickle.dump(dui2trail, f)
                print(f"Saved at {c} with size {len(dui2trail)}")
    except Exception as e:
        print(dui, dui2heading[dui], e)
    c += 1

with open(PICKLES_ADDRESS + 'dui2trail.pkl', 'wb') as f:
    pickle.dump(dui2trail, f)
print(f"Saved at {c} with size {len(dui2trail)}")

  0%|          | 1/29655 [00:01<9:05:50,  1.10s/it]

Saved at 0 with size 6


  0%|          | 54/29655 [00:47<6:10:09,  1.33it/s] 

D005061 Europe, Eastern 'result'


  0%|          | 59/29655 [00:52<8:31:55,  1.04s/it]

...

Saved at 29600 with size 29395


100%|█████████▉| 29622/29655 [1:59:07<00:14,  2.21it/s]

D007597 Jordan 'result'


100%|██████████| 29655/29655 [1:59:13<00:00,  4.15it/s]

Saved at 29655 with size 29432


In [7]:
#read dui2trail
with open(PICKLES_ADDRESS + 'dui2trail.pkl', 'rb') as f:
    dui2trail = pickle.load(f)
print(f"Loaded with size {len(dui2trail)}")

Loaded with size 29432


In [9]:
dui2trail['D001185']

['U000010', 'D007254', 'D003205', 'D000465', 'D001185']

In [10]:
dui2children = {k:[] for k in dui2trail.keys()}

for _, trail in dui2trail.items():
    if len(trail) >= 2:
        for i, elem in enumerate(trail):
            dui2children[elem].extend(trail[i+1:])

for k, v in dui2children.items():
    dui2children[k] = list(set(v))

with open(PICKLES_ADDRESS + 'dui2children.pkl', 'wb') as f:
    pickle.dump(dui2children, f)
print('dui2children.pkl saved')

children_print = [len(v) for k, v in dui2children.items()]
print('len(dui2children)', len(dui2children))
print('max(children_print)', max(children_print))
print('min(children_print)', min(children_print))
print('np.mean(children_print)', np.mean(children_print))

dui2children.pkl saved
len(dui2children) 29432
max(children_print) 10060
min(children_print) 0
np.mean(children_print) 5.076617287306333


In [20]:
dui2children = {k:[] for k in dui2trail.keys()}

for trail in tqdm(dui2trail.values()):
    #traverse the trail list from the end with enunmerator named as idx1
    for idx1, child_dui in enumerate(trail[::-1]):
        #select all elements from the beginning to the idx1, excluding idx1 itself
        for parent_dui in trail[:len(trail)-idx1-1]:
            dui2children[parent_dui].append(child_dui)

for k, v in dui2children.items():
    dui2children[k] = list(set(v))

with open(PICKLES_ADDRESS + 'dui2children.pkl', 'wb') as f:
    pickle.dump(dui2children, f)
print('dui2children.pkl saved')

children_print = [len(v) for k, v in dui2children.items()]
print('len(dui2children)', len(dui2children))
print('max(children_print)', max(children_print))
print('min(children_print)', min(children_print))
print('np.mean(children_print)', np.mean(children_print))

100%|██████████| 29432/29432 [00:00<00:00, 336159.26it/s]

dui2children.pkl saved
len(dui2children) 29432
max(children_print) 10060
min(children_print) 0
np.mean(children_print) 5.076617287306333


In [21]:
[get_name(each) for each in dui2children['D001185']]

['Computer Heuristics',
 'Machine Learning',
 'Expert Systems',
 'Sentiment Analysis',
 'Fuzzy Logic',
 'Deep Learning',
 'Natural Language Processing',
 'Neural Networks, Computer',
 'Unsupervised Machine Learning',
 'Supervised Machine Learning',
 'Robotics',
 'Support Vector Machine',
 'Robotic Surgical Procedures',
 'Knowledge Bases']

In [7]:
with open(PICKLES_ADDRESS + 'dui2trail.pkl', 'rb') as f:
    dui2trail = pickle.load(f)
len(dui2trail)

29432

In [10]:
max_depth = max([len(v) for v in dui2trail.values()])
print('max_depth', max_depth)

max_depth 14


In [26]:
level2duis = {level: [] for level in range(max_depth)}

for dui, trail in dui2trail.items():
    for i, ui in enumerate(trail):
        level2duis[i] += [ui]

for level, duis in level2duis.items():
    level2duis[level] = set(duis)

for level, duis in level2duis.items():
    print('level', level, '->', len(duis), 'unique topics.')

with open(PICKLES_ADDRESS + 'level2duis.pkl', 'wb') as f:
    pickle.dump(level2duis, f)
print('level2duis.pkl saved')

level 0 -> 14 unique topics.
level 1 -> 106 unique topics.
level 2 -> 1359 unique topics.
level 3 -> 4436 unique topics.
level 4 -> 6656 unique topics.
level 5 -> 6339 unique topics.
level 6 -> 4470 unique topics.
level 7 -> 2652 unique topics.
level 8 -> 1830 unique topics.
level 9 -> 1000 unique topics.
level 10 -> 418 unique topics.
level 11 -> 86 unique topics.
level 12 -> 44 unique topics.
level 13 -> 22 unique topics.
level2duis.pkl saved


In [27]:
dui2level = {}
for level, duis in level2duis.items():
    for dui in duis:
        dui2level[dui] = level

print('len(dui2level)', len(dui2level))

with open(PICKLES_ADDRESS + 'dui2level.pkl', 'wb') as f:
    pickle.dump(dui2level, f)
print('dui2level.pkl saved')

len(dui2level) 29432
dui2level.pkl saved
